In [1]:
import openai
import snowflake.connector
from tqdm import tqdm
import os

api_key_file = 'apikey.txt'
if os.path.isfile(api_key_file):
    with open(api_key_file) as f:
        openai.api_key = f.readline()
else:
    print(f"Error: {api_key_file} not found.")

OPENAI_API_KEY = openai.api_key

#login to snowflake db
con = snowflake.connector.connect(user='vishal.kumar@scale.com',
                                 account='pxa65918',
                                 authenticator='externalbrowser',
                                 warehouse='COMPUTE_WH',
                                 database='SCALE_CRAWLER',
                                 role='GENERAL_RO')

cs = con.cursor()

Initiating login request with your identity provider. A browser window should have opened for you to complete the login. If you can't see it, check existing browser windows, or your OS settings. Press CTRL+C to abort and try again...


In [3]:
#Get the prompt and response data from the snowflake db
sql = f'''
select
  ta.task,
  ta.attempted_by,
  t.params:templateVariables:Domain::string as Domain,
  t.params:templateVariables:Subtopic::string as Subtopic,
  t.params:templateVariables:Category::string as Category,
  t.params:templateVariables:Subcategory::string as Subcategory,
  t.params:templateVariables:Subcategory_Description::string as Subcategory_Description,
  ta.response :responses [0] :output :: string as Prompt,
  ta.response :rewrite :: string as Response
from
  scale_prod.public.taskattempts ta
  join scale_prod.public.tasks t on ta.task = t._id
where
  ta.project = '65c6b841652f25eabae60e72'
limit
  10
'''
cs.execute(sql)
idf = cs.fetch_pandas_all()

In [4]:
def evaluator_gpt(prompt):
    completion = openai.ChatCompletion.create(
        model="gpt-4-turbo-preview",
        messages=[{"role":"user", "content":prompt}],
        temperature=0.1
    )
    response = completion.choices[0].message
    response = response.content
    return response

In [6]:
#start a for loop to iterate through idf, add tqdm to show progress
for i in tqdm(range(len(idf))):
    #get the domain, category, subcategory and subcategory description from the first row of the dataframe
    domain = idf['DOMAIN'][i]
    category = idf['CATEGORY'][i]
    subcategory = idf['SUBCATEGORY'][i]
    subcategory_description = idf['SUBCATEGORY_DESCRIPTION'][i]
    prompt = idf['PROMPT'][i]
    eval_query="You are given a prompt(question) and you need to check whether it follows various criteria. Here are the criterion you need to check for\n\nDomain\n Check whether the question belongs to this domain or not. \nCategory\n Check whether the question belongs to this category or not\nSubcategory\n Check whether the question belongs to this category or not based on a subcategory description which will be provided to you. \n\nHere is the question and the criteria that needs your assessment\n\n" + prompt + "\nDomain:" + domain + "\nCategory:" + category + "\nSubcategory:" + subcategory + " (" + subcategory_description + ")\nRespond in following format:\nDomain: Yes/No\nCategory: Yes/No\nSubcategory: Yes/No\n\nDont say anything else, just stick to the format above"
    response = evaluator_gpt(eval_query)
    print(response)

  0%|          | 0/10 [00:00<?, ?it/s]

 10%|█         | 1/10 [00:01<00:11,  1.31s/it]

Domain: Yes
Category: Yes
Subcategory: Yes


 20%|██        | 2/10 [00:02<00:08,  1.05s/it]

Domain: Yes
Category: Yes
Subcategory: No


...

Domain: Yes
Category: No
Subcategory: No


 90%|█████████ | 9/10 [00:11<00:01,  1.18s/it]

Domain: Yes
Category: Yes
Subcategory: Yes


100%|██████████| 10/10 [00:12<00:00,  1.22s/it]

Domain: Yes
Category: Yes
Subcategory: Yes
